# Library

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# 수정사항
- 2023.03.21
    - item 2 : EG
        - 1. 개별 차량에 대한 지역코드를 부여하여, 지역마다 얼마나 차량들이 존재하는지 구분이 필요한 것 같음

    이 외에도 내일 오전에 동양에서 가시화와 관련된 회의에서 추가적으로 필요하다고 판단되는 사항에 대해 추가적으로 분석이 필요할 것 같음

- 2023.03.22
    - result파일에 'EG' 컬럼 추가

- 2023.03.24
    - result파일에 '배인번호_수정', '등급_수정', 'DPF유무_수정' 컬럼 추가

# fold path

In [3]:
raw_fold = 'D:/data/big2/BD1/raw'
ods_fold = 'D:/data/big2/ODS'
sample_fold = 'D:/data/big2/BD1/sample'
analysis_fold = 'D:/data/big2/BD1/an'

# Load

## 병합 result 최종 파일

In [4]:
# about 11.4s
name = '4등급_차량_리스트_양식_result_최종(2023.03.31_PM.07.30.15)'
file_name = f'{name}.csv'
result = pd.read_csv(os.path.join(raw_fold, file_name), low_memory=False, encoding='cp949')
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1153813 entries, 0 to 1153812
Data columns (total 31 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   차대번호      1153813 non-null  object 
 1   차량번호      1153813 non-null  object 
 2   제원관리번호    1153813 non-null  object 
 3   차종        1153813 non-null  object 
 4   용도        1153813 non-null  object 
 5   최초등록일     1153813 non-null  int64  
 6   연식        1153813 non-null  int64  
 7   제작일자      1153809 non-null  float64
 8   검사유효일     1153810 non-null  float64
 9   배출가스인증번호  1146663 non-null  object 
 10  등급        1153813 non-null  int64  
 11  차명        1153813 non-null  object 
 12  차종분류      1153813 non-null  object 
 13  차종유형      1153813 non-null  object 
 14  자동차형식     1153812 non-null  object 
 15  제작사명      1153795 non-null  object 
 16  연료        1153813 non-null  object 
 17  엔진형식      1153813 non-null  object 
 18  총중량       1153813 non-null  int64  
 19  적재중량      1153589 non

In [5]:
result.loc[result['배출가스인증번호'] == '9MY-AD-14-04', '제원관리번호'].value_counts()

02420011500011209    584
02420011500031210    208
02420011500001209    184
Name: 제원관리번호, dtype: int64

In [6]:
result.loc[result['배출가스인증번호'] == '9MY-AD-14-04', ['제원관리번호', '등급_수정']].head()

,제원관리번호,등급_수정
1005471,02420011500001209,4
1005472,02420011500011209,4
1005473,02420011500011209,4
1005474,02420011500011209,4
1005475,02420011500001209,4


## 병합 result 원본 파일

In [7]:
# about 10.3s
name = '4등급_차량_리스트_양식_result'
file_name = f'{name}.csv'
rw = pd.read_csv(os.path.join(raw_fold, file_name), low_memory=False, encoding='cp949')
rw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1502235 entries, 0 to 1502234
Data columns (total 28 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   차대번호    1502235 non-null  object 
 1   차량번호    1502235 non-null  object 
 2   제원관리번호  1502235 non-null  object 
 3   차종      1502235 non-null  object 
 4   용도      1502235 non-null  object 
 5   최초등록일   1502235 non-null  int64  
 6   연식      1502235 non-null  int64  
 7   제작일자    1335178 non-null  float64
 8   검사유효일   1502223 non-null  float64
 9   인증번호    1382225 non-null  object 
 10  등급      1502235 non-null  int64  
 11  차명      1502235 non-null  object 
 12  차종분류    1502235 non-null  object 
 13  차종유형    1502235 non-null  object 
 14  자동차형식   1501958 non-null  object 
 15  제작사명    1492590 non-null  object 
 16  연료      1502235 non-null  object 
 17  엔진형식    1500400 non-null  object 
 18  총중량     1502234 non-null  float64
 19  적재중량    1501728 non-null  float64
 20  엔진출력    1501882 non-null

In [8]:
rw.loc[rw['인증번호'] == '9MY-AD-14-04', '제원관리번호'].value_counts()

02420011500011209    584
02420011500031210    208
02420011500001209    184
Name: 제원관리번호, dtype: int64

# 전처리

## 컬럼명 수정

In [9]:
result.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '배인번호_수정', '등급_수정', 'DPF유무_수정'],
      dtype='object')

In [10]:
rw.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무'],
      dtype='object')

In [11]:
rw = rw.rename(columns={'인증번호':'배출가스인증번호'})

## 9MY-AD-14-04

In [12]:
result.loc[result['배출가스인증번호'] == '9MY-AD-14-04', '등급_수정'].shape

(976,)

In [13]:
result.loc[result['배출가스인증번호'] == '9MY-AD-14-04', '등급_수정'] = 4

## 유니목1200 5등급으로 수정

In [14]:
result.loc[result['차대번호'] =='WDB1260251A480232', '등급_수정']

1153460    5
Name: 등급_수정, dtype: int64

In [15]:
result.loc[result['차대번호'] =='WDB1260251A480232', '등급_수정'] = 5

## 4등급 차량만

In [16]:
result['등급_수정'].value_counts(dropna=False)

4    1149290
3       4396
5        127
Name: 등급_수정, dtype: int64

In [17]:
result1 = result.loc[result['등급_수정'] == 4]
result1.shape

(1149290, 31)

In [18]:
g3_g5 = result.loc[result['등급_수정'] != 4]
g3_g5.shape

(4523, 31)

## 중복 차대번호 확인
- 최종등록일자, 배출가스검사유효일자 최신 데이터 사용

In [19]:
len(result1['차대번호'].unique())

1149290

## 연료 확인

In [20]:
result1['연료'].value_counts(dropna=False)

경유    1149290
Name: 연료, dtype: int64

## 원본 데이터에서 4등급 휘발유_가스 차량 추출

In [21]:
rw.shape

(1502235, 28)

In [22]:
# 연료 확인
rw['연료'].value_counts(dropna=False)

경유             1153813
휘발유             322983
LPG(액화석유가스)      25400
기타연료                37
알코올                  1
CNG(압축천연가스)          1
Name: 연료, dtype: int64

In [23]:
# 등급 확인
rw['등급'].value_counts()

4    1502235
Name: 등급, dtype: int64

In [24]:
# 348383
rw1 = rw.loc[(rw['연료'] == '휘발유') | (rw['연료'] == 'LPG(액화석유가스)')].reset_index(drop=True)
rw1.shape

(348383, 28)

In [25]:
# 39
rw_rest = rw.loc[(rw['연료'] == '알코올') | (rw['연료'] == 'CNG(압축천연가스)') | (rw['연료'] == '기타연료')].reset_index(drop=True)
rw_rest.shape

(39, 28)

In [26]:
result1.shape[0] + rw1.shape[0]

1497673

In [27]:
result1['fuel'] = '경유'
rw1['fuel'] = '휘발유_가스'

C:\Users\kbjung\AppData\Local\Temp\ipykernel_16036\3367011894.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result1['fuel'] = '경유'


In [28]:
result1['fuel'].value_counts()

경유    1149290
Name: fuel, dtype: int64

In [29]:
rw1['fuel'].value_counts()

휘발유_가스    348383
Name: fuel, dtype: int64

## 경유와 휘발유_가스 병합

In [30]:
result1.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', 'fuel'],
      dtype='object')

In [31]:
rw1.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', 'fuel'],
      dtype='object')

In [32]:
df = pd.concat([result1, rw1], ignore_index=True)
df.shape

(1497673, 32)

In [33]:
df['fuel'].value_counts()

경유        1149290
휘발유_가스     348383
Name: fuel, dtype: int64

In [34]:
df['DPF유무_수정'].value_counts(dropna=False)

무       879463
NaN     348383
유       251010
확인불가     18817
Name: DPF유무_수정, dtype: int64

In [35]:
# df['DPF유무_수정'] = df['DPF유무_수정'].fillna('무')
# df.loc[df['DPF유무_수정'] == '확인불가', 'DPF유무_수정'] = '무'

In [36]:
df['DPF유무_수정'].value_counts(dropna=False)

무       879463
NaN     348383
유       251010
확인불가     18817
Name: DPF유무_수정, dtype: int64

# 분석

In [37]:
grade_list = []
for f, y, cy, e in tqdm(df[['fuel', '제작일자', '연식', 'DPF유무_수정']].values):
    if (f == '휘발유_가스') and ( (19980101 <= y <= 20001231) or (1998 <= cy <= 2000) ):
        grade_list.append('A')
    elif (f == '휘발유_가스') and ( (y <= 19971231) or (cy <= 1997) ):
        grade_list.append('B')
    elif (f == '경유') and ( (y >= 20080101) or (cy >= 2008) ) and (e == '유'):
        grade_list.append('A')
    elif (f == '경유') and ( (y <= 20071231) or (cy <= 2007) )and (e == '유'):
        grade_list.append('B')
    elif (f == '경유') and ( (y >= 20080101) or (cy >= 2008) ) and (e == '무'):
        grade_list.append('C')
    elif (f == '경유') and ( (y <= 20071231) or (cy <= 2007) ) and (e == '무'):
        grade_list.append('D')
    else:
        grade_list.append('X')
len(grade_list)

100%|██████████| 1497673/1497673 [00:02<00:00, 525947.00it/s]


1497673

In [38]:
grade_list[:5]

['D', 'D', 'D', 'D', 'D']

In [39]:
num = 85420
grade_list[num:num+10]

['C', 'D', 'C', 'D', 'C', 'C', 'D', 'D', 'C', 'C']

In [40]:
grade_list[-5:]

['B', 'B', 'B', 'A', 'B']

In [41]:
df['Grade'] = grade_list

In [42]:
num_by_fuel = df.groupby(['fuel'])['차대번호'].count().reset_index().sort_values('차대번호', ascending=False)
num_by_fuel = num_by_fuel.rename(columns={'차대번호':'대수'})
num_by_fuel

,fuel,대수
0,경유,1149290
1,휘발유_가스,348383


In [43]:
num_by_grade = df.groupby(['fuel', 'Grade'])['차대번호'].count().unstack()
# num_by_grade = num_by_grade.rename('대수')
num_by_grade

Grade,A,B,C,D,X
fuel,,,,,
경유,165289.00,85721.00,840425.00,39038.00,18817.00
휘발유_가스,40729.00,291903.00,NaN,NaN,15751.00


In [44]:
num_by_grade_by_local = df.groupby(['시도명', 'Grade'])['차대번호'].count().unstack()
# num_by_grade_by_local = num_by_grade_by_local.rename('대수')
num_by_grade_by_local

Grade,A,B,C,D,X
시도명,,,,,
강원도,7469,10757,34963,1682,1054
경기도,51786,99665,189687,9851,8529
경상남도,15982,23194,71293,3030,2453
경상북도,14075,23448,73252,2905,2319
광주광역시,5677,10059,21859,1006,793
대구광역시,9970,18532,36183,1933,1873
대전광역시,5729,11700,21701,1097,1049
부산광역시,11348,19490,41171,1781,1778
서울특별시,25125,62526,77485,4308,5884


In [45]:
num_by_grade_by_local.columns

Index(['A', 'B', 'C', 'D', 'X'], dtype='object', name='Grade')

In [46]:
num_by_grade_by_local.index

Index(['강원도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시', '부산광역시',
       '서울특별시', '세종특별자치시', '울산광역시', '인천광역시', '전라남도', '전라북도', '제주특별자치도', '충청남도',
       '충청북도'],
      dtype='object', name='시도명')

In [47]:
df.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', 'fuel', 'Grade'],
      dtype='object')

## [출력] df1

In [48]:
df.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', 'fuel', 'Grade'],
      dtype='object')

In [49]:
df1 = df.drop('fuel', axis=1)
df1.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', 'Grade'],
      dtype='object')

In [50]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1497673 entries, 0 to 1497672
Data columns (total 32 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   차대번호      1497673 non-null  object 
 1   차량번호      1497673 non-null  object 
 2   제원관리번호    1497673 non-null  object 
 3   차종        1497673 non-null  object 
 4   용도        1497673 non-null  object 
 5   최초등록일     1497673 non-null  int64  
 6   연식        1497673 non-null  int64  
 7   제작일자      1330652 non-null  float64
 8   검사유효일     1497661 non-null  float64
 9   배출가스인증번호  1377718 non-null  object 
 10  등급        1497673 non-null  int64  
 11  차명        1497673 non-null  object 
 12  차종분류      1497673 non-null  object 
 13  차종유형      1497673 non-null  object 
 14  자동차형식     1497398 non-null  object 
 15  제작사명      1488034 non-null  object 
 16  연료        1497673 non-null  object 
 17  엔진형식      1495841 non-null  object 
 18  총중량       1497672 non-null  float64
 19  적재중량      1497331 non

In [51]:
df1[['배출가스인증번호', '제원관리번호']].head()

,배출가스인증번호,제원관리번호
0,4MY-AD-20,02420000100031305
1,4MY-AD-20,02420000100031305
2,4MY-AD-20,02420000100031305
3,4MY-AD-20,02420000100031305
4,4MY-AD-20,02420000100031305


In [52]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
today_date

'2023.04.13_PM.01.23.01'

In [53]:
# about 20.8s
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
df1.to_csv(os.path.join(raw_fold, f'4등급 차량 리스트 양식(경유, 휘발유, LPG) result 최종 Grade 추가({today_date}).csv'), index=False, encoding='cp949')

In [100]:
# # about 20.6s
# today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
# df1.to_csv(os.path.join(raw_fold, f'4등급 차량 리스트 양식 result 최종 Grade 추가({today_date}).txt'), index=False, encoding='cp949')

In [101]:
# # about 20.5s
# today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
# df1.iloc[:500000].to_csv(os.path.join(raw_fold, f'4등급 차량 리스트 양식 result 최종 Grade 추가({today_date})01.csv'), index=False, encoding='cp949')
# df1.iloc[500000:].to_csv(os.path.join(raw_fold, f'4등급 차량 리스트 양식 result 최종 Grade 추가({today_date})02.csv'), index=False, encoding='cp949')

In [102]:
# # about 20.5s
# today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
# df1.iloc[:500000].to_csv(os.path.join(raw_fold, f'4등급 차량 리스트 양식 result 최종 Grade 추가({today_date})01.txt'), index=False, encoding='cp949')
# df1.iloc[500000:].to_csv(os.path.join(raw_fold, f'4등급 차량 리스트 양식 result 최종 Grade 추가({today_date})02.txt'), index=False, encoding='cp949')

In [103]:
# # about 12m 39.8s
# today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
# df1.iloc[:500000].to_excel(os.path.join(raw_fold, f'4등급 차량 리스트 양식 result 최종 Grade 추가({today_date})01.xlsx'), index=False)
# df1.iloc[500000:].to_excel(os.path.join(raw_fold, f'4등급 차량 리스트 양식 result 최종 Grade 추가({today_date})02.xlsx'), index=False)

In [104]:
# # about 12m 35.7s
# # xlsxwriter
#     # 속도 차이 없음
#     # 기본과 파일 용량 차이 없음
# today_date = datetime.today().strftime("%Y.%m.%d_%p.%H.%M.%S")
# df1.iloc[:500000].to_excel(os.path.join(raw_fold, f'4등급 차량 리스트 양식 result 최종 Grade 추가({today_date})01.xlsx'), index=False, engine='xlsxwriter')
# df1.iloc[500000:].to_excel(os.path.join(raw_fold, f'4등급 차량 리스트 양식 result 최종 Grade 추가({today_date})02.xlsx'), index=False, engine='xlsxwriter')

In [105]:
# # about 
# # openpyxl
#     # 속도 조금 느림
#     # 기본보다 용량 큼
# today_date = datetime.today().strftime("%Y.%m.%d_%p.%H.%M.%S")
# df1.iloc[:500000].to_excel(os.path.join(raw_fold, f'4등급 차량 리스트 양식 result 최종 Grade 추가({today_date})01.xlsx'), index=False, engine='openpyxl')
# df1.iloc[500000:].to_excel(os.path.join(raw_fold, f'4등급 차량 리스트 양식 result 최종 Grade 추가({today_date})02.xlsx'), index=False, engine='openpyxl')

## [출력] 4등급 경유, 4등급 휘발유_가스, 3등급 경유, 5등급 경유 병합

In [54]:
rw.shape

(1502235, 28)

In [55]:
df.shape[0] + g3_g5.shape[0] + rw_rest.shape[0]

1502235

In [56]:
total = pd.concat([df, g3_g5, rw_rest], ignore_index=True)
total.shape

(1502235, 33)

In [57]:
total.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', 'fuel', 'Grade'],
      dtype='object')

In [59]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
today_date

'2023.04.13_PM.01.24.45'

In [58]:
# about 21.1s
total.to_csv(os.path.join(raw_fold, f'4등급 차량 리스트 양식(전체) result 최종 Grade 추가({today_date}).csv'), index=False, encoding='cp949')

In [60]:
# about 13m 7.6s
total.iloc[:700000].to_excel(os.path.join(raw_fold, f'4등급 차량 리스트 양식(전체) result 최종 Grade 추가01({today_date}).xlsx'), index=False)
total.iloc[700000:].to_excel(os.path.join(raw_fold, f'4등급 차량 리스트 양식(전체) result 최종 Grade 추가02({today_date}).xlsx'), index=False)

## [출력] EG 통계

In [106]:
analysis_fold

'D:/data/big2/BD1/an'

In [107]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
with pd.ExcelWriter(os.path.join(analysis_fold, f'G4_통계_EG_{today_date}.xlsx')) as writer:
    num_by_fuel.to_excel(writer, sheet_name='연료별', index=False)
    num_by_grade.to_excel(writer, sheet_name='등급별')
    num_by_grade_by_local.to_excel(writer, sheet_name='지역별')

## [출력] 지역별 샘플

In [54]:
df.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', 'fuel', 'Grade'],
      dtype='object')

In [55]:
df[['법정동코드', '시도명', '시군구명', 'Grade', '차량번호', 'DPF유무_수정', '연식']].head(10)

,법정동코드,시도명,시군구명,Grade,차량번호,DPF유무_수정,연식
0,3017011600,대전광역시,서구,D,21로7811,무,2006
1,2635010700,부산광역시,해운대구,D,23거2347,무,2006
2,4148012200,경기도,파주시,D,04어9316,무,2006
3,2623010900,부산광역시,부산진구,D,14우2622,무,2006
4,5013010300,제주특별자치도,서귀포시,D,07머3269,무,2006
5,4150031021,경기도,이천시,D,58거7827,무,2006
6,4817012100,경상남도,진주시,D,64보3244,무,2006
7,4167025035,경기도,여주시,D,69가2031,무,2006
8,4213010300,강원도,원주시,D,06주6090,무,2006
9,4684035021,전라남도,무안군,D,32어5631,무,2006


In [74]:
# df[['법정동코드', '시도명', '시군구명', 'Grade', '차량번호', 'DPF유무_수정', '연식']].head(10).to_excel(os.path.join(analysis_fold, f'샘플_법정동코드_지역_{today_date}.xlsx'), index=False)

In [56]:
df.groupby(['시도명', '시군구명', 'Grade', 'DPF유무_수정'], dropna=False)['차대번호'].count()

시도명   시군구명  Grade  DPF유무_수정
강원도   강릉시   A      유            928
                   NaN          117
            B      유            459
                   NaN          981
            C      무           4221
                               ... 
충청북도  충주시   B      NaN         1425
            C      무           4999
            D      무            218
            X      확인불가         103
                   NaN           49
Name: 차대번호, Length: 2006, dtype: int64

# LPG 만 추출

In [11]:
rw['연료'].value_counts()

경유             1153813
휘발유             322983
LPG(액화석유가스)      25400
기타연료                37
알코올                  1
CNG(압축천연가스)          1
Name: 연료, dtype: int64

In [12]:
# 25400
rw_lpg = rw.loc[rw['연료'] == 'LPG(액화석유가스)']
rw_lpg.shape

(25400, 28)

In [13]:
rw_lpg.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무'],
      dtype='object')

In [14]:
grade_list = []
for f, y, cy in tqdm(rw_lpg[['연료', '제작일자', '연식']].values):
    if (f == 'LPG(액화석유가스)') and ( (19980101 <= y <= 20001231) or (1998 <= cy <= 2000) ):
        grade_list.append('A')
    elif (f == 'LPG(액화석유가스)') and ( (y <= 19971231) or (cy <= 1997) ):
        grade_list.append('B')
    else:
        grade_list.append('X')
len(grade_list)

100%|██████████| 25400/25400 [00:00<00:00, 528947.53it/s]


25400

In [15]:
rw_lpg['Grade'] = grade_list

C:\Users\kbjung\AppData\Local\Temp\ipykernel_7388\3232022228.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rw_lpg['Grade'] = grade_list


In [25]:
rw_lpg['Grade'].isnull().sum()

0

## [출력] 샘플

In [22]:
rw_lpg.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', 'Grade'],
      dtype='object')

In [23]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%H.%M.%S")
today_date

'2023.03.30_AM.09.54.55'

In [24]:
rw_lpg.to_csv(os.path.join(raw_fold, f'4등급 차량 리스트 양식 result(LPG) Grade 추가({today_date}).csv'), index=False, encoding='cp949')

In [16]:
num_by_fuel = rw_lpg.groupby(['연료'])['차대번호'].count().reset_index().sort_values('차대번호', ascending=False)
num_by_fuel = num_by_fuel.rename(columns={'차대번호':'대수'})
num_by_fuel

,연료,대수
0,LPG(액화석유가스),25400


In [17]:
num_by_grade = rw_lpg.groupby(['연료', 'Grade'])['차대번호'].count().unstack()
# num_by_grade = num_by_grade.rename('대수')
num_by_grade

Grade,A,B,X
연료,,,
LPG(액화석유가스),11806,11832,1762


In [18]:
num_by_grade_by_local = rw_lpg.groupby(['시도명', 'Grade'])['차대번호'].count().unstack()
# num_by_grade_by_local = num_by_grade_by_local.rename('대수')
num_by_grade_by_local

Grade,A,B,X
시도명,,,
강원도,266,449,48
경기도,3067,3239,404
경상남도,499,408,99
경상북도,618,722,86
광주광역시,320,312,53
대구광역시,795,738,79
대전광역시,564,594,45
부산광역시,725,656,76
서울특별시,2237,1524,450


## [출력] LPG EG 통계

In [19]:
analysis_fold

'D:/data/big2/BD1/an'

In [20]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%H.%M.%S")
today_date

'2023.03.30_AM.09.51.45'

In [21]:
with pd.ExcelWriter(os.path.join(analysis_fold, f'G4_LPG_통계_EG_{today_date}.xlsx')) as writer:
    num_by_fuel.to_excel(writer, sheet_name='연료별', index=False)
    num_by_grade.to_excel(writer, sheet_name='등급별')
    num_by_grade_by_local.to_excel(writer, sheet_name='지역별')

# 배인번호 HB 포함한 샘플 출력

In [108]:
df1.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', 'Grade'],
      dtype='object')

In [114]:
hb = df1.copy()

In [115]:
hb['배출가스인증번호'] = hb['배출가스인증번호'].astype('str')
hb['배인번호_수정'] = hb['배인번호_수정'].astype('str')

In [116]:
temp = hb.loc[hb['배인번호_수정'].str.contains('HB')].reset_index(drop=True)
temp.shape

(2672, 32)

## [출력] 샘플

In [117]:
# about 1.5s
temp.to_excel(os.path.join(raw_fold, '샘플_배인번호_수정에_HB포함.xlsx'), index=False)

# 코드 마지막